In [4]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import KFold

In [2]:
path = '/home/kai/data/kaggle/talkingdata/data/'
train = pd.read_csv(path + 'train_cleaned.csv')
test = pd.read_csv(path + 'test_cleaned.csv')

# Get K Fold

In [25]:
K = 6
kf = KFold(n_splits=K, shuffle = False)

history_index = []
train_index = []
for h,t in kf.split(train):
    history_index.append(h)
    train_index.append(t)
    
### use last fold as example

In [26]:
import sys
print(sys.getsizeof(train)/ 1024 **3)

19.286978788673878


# use last fold as example

In [27]:
df_history = train.iloc[history_index[-1]]
df_train = train.iloc[train_index[-1]]

In [28]:
import sys
print(sys.getsizeof(df_train)/ 1024 **3)
print(len(df_train))

3.4441033601760864
30817315


In [31]:
orders = {}
feature_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel',
              'year', 
              'month',
              'week',
              'day',
              'hour',
              'timestamp',
              'minute',
              'second']
for col in feature_col:
    orders[col] = 10 ** (int(np.log(max(train[col].max(),test[col].max() ) + 1) / np.log(10)) + 1)
def get_group(df, cols):
    """
    define an encoding method which can ganrantee the adding value will be unique.
    eg: artist_name_composer will be a combination of (artist_name,composer) and the encoding will reflect the unqiue combination of those two
    """
    group = df[cols[0]].copy()
    for col in cols[1:]:
        group = group * orders[col] + df[col]
        
    return group

In [32]:
orders

{'app': 1000,
 'channel': 1000,
 'day': 100,
 'device': 10000,
 'hour': 100,
 'ip': 1000000,
 'minute': 100,
 'month': 100,
 'os': 1000,
 'second': 100,
 'timestamp': 10000000000,
 'week': 100,
 'year': 10000}